In [ ]:
import holodeck as holo
from holodeck import single_sources, utils, plot, detstats
from holodeck.constants import YR
import holodeck.anisotropy as anis

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py

In [ ]:
nn=615
filename = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss60_anis09B_2023-06-16_uniform-09b_n1000_r50_f40_l2000/sam-lib__p000615.npz'
file = np.load(filename)
print(file.files)
hc_ss = file['hc_ss']
hc_bg = file['hc_bg']
fobs = file['fobs']


file.close()

shape=hc_ss.shape
nfreqs = shape[-3]
nreals = shape[-2]
nloudest = shape[-1]
print('nfreqs=%d, nreals=%d, nloudest=%d' % (nfreqs, nreals, nloudest))


# sspath = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss51-2023-05-22_uniform_07a_n1000_r100_f40_l2000/'
# hdfname = sspath+'sam_lib.hdf5'
# ssfile = h5py.File(hdfname, 'r')
# dfobs = ssfile['dfobs'][:]

# 1 hc


## 1.0A Calculate and save NPZ

In [ ]:
# place single sources at random pixels, for one map
def make_ss_map(hc_ss, nside, pix_ss=None):
    npix = hp.nside2npix(nside)
    if pix_ss is None:
        pix_ss = np.random.randint(0, npix-1, size=hc_ss.size).reshape(hc_ss.shape)
    moll_ss = np.zeros((len(hc_ss), len(hc_ss[0]), npix,))
    for ff in range(hc_ss.shape[-3]):
        for rr in range(hc_ss.shape[-2]):
            for ll in range(hc_ss.shape[-1]):
                moll_ss[ff,rr,pix_ss[ff,rr,ll]] = np.sqrt(moll_ss[ff,rr,pix_ss[ff,rr,ll]]**2
                                                         + hc_ss[ff,rr,ll]**2)
    return moll_ss, pix_ss
moll_ss08, pix_ss08 = make_ss_map(hc_ss, 8)
print(f"{moll_ss08.shape=}")
print(f"{pix_ss08.shape=}")


def convert_pix_ss(pix_ss1, nside1, nside2):
    theta, phi = hp.pix2ang(nside1, pix_ss1)
    print(f"{theta.shape=}, {phi.shape=}")
    pix_ss2 = hp.ang2pix(nside2, theta, phi).reshape(pix_ss1.shape)
    return pix_ss2

pix_ss16 = convert_pix_ss(pix_ss08, 8, 16)
pix_ss32 = convert_pix_ss(pix_ss08, 8, 32)
pix_ss64 = convert_pix_ss(pix_ss08, 8, 64)
# pix_ss128 = convert_pix_ss(pix_ss08, 8, 128)

In [ ]:
save_loc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/healpix/varying_nside_testnpz'

np.savez(save_loc+'/pix_ss_B684.npz', 
         pix_ss08=pix_ss08, pix_ss16=pix_ss16, pix_ss32=pix_ss32, pix_ss64=pix_ss64,) #pix_ss128=pix_ss128,
        #  moll2dOm_ss08=moll2dOm_ss08, moll2dOm_ss16=moll2dOm_ss16, moll2dOm_ss32=moll2dOm_ss32, moll2dOm_ss64=moll2dOm_ss64, #moll2dOm_ss128=moll2dOm_ss128,
        #  moll2dOm_bg08=moll2dOm_bg08, moll2dOm_bg16=moll2dOm_bg16, moll2dOm_bg32=moll2dOm_bg32, moll2dOm_bg64=moll2dOm_bg64,# moll_bg128=moll_bg128,
        #  moll2dOm_tt08=moll2dOm_tt08, moll2dOm_tt16=moll2dOm_tt16, moll2dOm_tt32=moll2dOm_tt32, moll2dOm_tt64=moll2dOm_tt64, #moll_tt128=moll_tt128,
        #  Cl2dOm_arrs=Cl2dOm_arrs, nsides=nsides)
        #  Cl_08=Cl_08, Cl_16=Cl_16, Cl_32=Cl_32, Cl_64=Cl_64, Cl_128=Cl_128)

## 1.0 Load NPZ

In [ ]:
save_loc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/healpix/varying_nside_testnpz'
hcfile = np.load(save_loc+'/pix_ss_B684.npz')

# pix_ss08=pix_ss08, pix_ss16=pix_ss16, pix_ss32=pix_ss32, pix_ss64=pix_ss64, pix_ss128=pix_ss128,
# moll_ss08=moll_ss08, moll_ss16=moll_ss16, moll_ss32=moll_ss32, moll_ss64=moll_ss64, moll_ss128=moll_ss128,
# moll_bg08=moll_bg08, moll_bg16=moll_bg16, moll_bg32=moll_bg32, moll_bg64=moll_bg64, moll_bg128=moll_bg128,
# moll_tt08=moll_tt08, moll_tt16=moll_tt16, moll_tt32=moll_tt32, moll_tt64=moll_tt64, moll_tt128=moll_tt128,
pix_ss08 = hcfile['pix_ss08']
pix_ss16 = hcfile['pix_ss16']
pix_ss32 = hcfile['pix_ss32']
pix_ss64 = hcfile['pix_ss64']
# pix_ss128 = hcfile['pix_ss128']
# Cl_arrs = hcfile['Cl_arrs']
# nsides=hcfile['nsides']

hcfile.close()

lmax=6
nsides=np.array([8,16,32,64])

# 3 hc^2/dOmega 


## 3.0 Load NPZ

In [ ]:
# save_loc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/healpix/varying_nside_testnpz'
# hcfile = np.load(save_loc+'/hc2dOm_A.npz')

# # pix_ss08=pix_ss08, pix_ss16=pix_ss16, pix_ss32=pix_ss32, pix_ss64=pix_ss64, pix_ss128=pix_ss128,
# # moll_ss08=moll_ss08, moll_ss16=moll_ss16, moll_ss32=moll_ss32, moll_ss64=moll_ss64, moll_ss128=moll_ss128,
# # moll_bg08=moll_bg08, moll_bg16=moll_bg16, moll_bg32=moll_bg32, moll_bg64=moll_bg64, moll_bg128=moll_bg128,
# # moll_tt08=moll_tt08, moll_tt16=moll_tt16, moll_tt32=moll_tt32, moll_tt64=moll_tt64, moll_tt128=moll_tt128,
# # pix_ss08 = hcfile['pix_ss08']
# # pix_ss16 = hcfile['pix_ss16']
# # pix_ss32 = hcfile['pix_ss32']
# # pix_ss64 = hcfile['pix_ss64']
# # # pix_ss128 = hcfile['pix_ss128']
# Cl2dOm_arrs = hcfile['Cl2dOm_arrs']
# # nsides=hcfile['nsides']

# hcfile.close()

# lmax=6

## 3.1 Maps

In [ ]:
# spread background evenly across pixels

def make_hc2dOm_bg_map(hc_bg, nside):
    npix = hp.nside2npix(nside)
    area = hp.nside2pixarea(nside)
    moll_bg = (np.ones((hc_bg.shape + (npix,))) 
            * hc_bg[:,:,np.newaxis]**2/npix/area) # (frequency, realization, pixel)
    return moll_bg

nsides=np.array([8,16,32, 64]) #, 128])
moll2dOm_bg08 = make_hc2dOm_bg_map(hc_bg, nside=8)
moll2dOm_bg16 = make_hc2dOm_bg_map(hc_bg, nside=16)
moll2dOm_bg32 = make_hc2dOm_bg_map(hc_bg, nside=32)
moll2dOm_bg64 = make_hc2dOm_bg_map(hc_bg, nside=64)
# moll2dOm_bg128 = make_hc2dOm_bg_map(hc_bg, nside=128)
print(moll2dOm_bg08.shape)


In [ ]:
# place single sources at random pixels, for one map

def make_hc2dOm_ss_map(hc_ss, nside, pix_ss=None):
    npix = hp.nside2npix(nside)
    area = hp.nside2pixarea(nside)
    if pix_ss is None:
        pix_ss = np.random.randint(0, npix-1, size=hc_ss.size).reshape(hc_ss.shape)
    moll_ss = np.zeros((len(hc_ss), len(hc_ss[0]), npix,))
    for ff in range(hc_ss.shape[-3]):
        for rr in range(hc_ss.shape[-2]):
            for ll in range(hc_ss.shape[-1]):
                moll_ss[ff,rr,pix_ss[ff,rr,ll]] = (moll_ss[ff,rr,pix_ss[ff,rr,ll]] 
                                                   + hc_ss[ff,rr,ll]**2 /area)
    return moll_ss, pix_ss

In [ ]:
moll2dOm_ss08, temp = make_hc2dOm_ss_map(hc_ss, 8, pix_ss=pix_ss08)
moll2dOm_ss16, temp = make_hc2dOm_ss_map(hc_ss, 16, pix_ss16)
moll2dOm_ss32, temp = make_hc2dOm_ss_map(hc_ss, 32, pix_ss32)
moll2dOm_ss64, temp = make_hc2dOm_ss_map(hc_ss, 64, pix_ss64)
# moll2dOm_ss128, temp = make_hc2dOm_ss_map(hc_ss, 128, pix_ss128)

In [ ]:
moll2dOm_tt08 = (moll2dOm_ss08 + moll2dOm_bg08)
moll2dOm_tt16 = (moll2dOm_ss16 + moll2dOm_bg16)
moll2dOm_tt32 = (moll2dOm_ss32 + moll2dOm_bg32)
moll2dOm_tt64 = (moll2dOm_ss64 + moll2dOm_bg64)
# moll2dOm_tt128 = (moll2dOm_ss128 + moll2dOm_bg128)

In [ ]:

rr=0
for ff in range(0,5):
    fig, axs = plot.figax(ncols=3, figsize=(15,5))
    plt.axes(axs[0])
    hp.mollview(moll2dOm_tt08[ff,rr], unit='$h_c^2/d\Omega$',
                title='f=%.2f yr$^{-1}$, nside=%d' % (fobs[ff]*YR,  8), hold=True )
    plt.axes(axs[1])
    hp.mollview(moll2dOm_tt16[ff,rr], unit='$h_c^2/d\Omega$',
                title='f=%.2f yr$^{-1}$, nside=%d' % (fobs[ff]*YR, 16), hold=True )
    plt.axes(axs[2])
    hp.mollview(moll2dOm_tt32[ff,rr], unit='$h_c^2/d\Omega$',
                title='f=%.2f yr$^{-1}$, nside=%d' % (fobs[ff]*YR, 32), hold=True )
    fig.tight_layout()


## 3.2 sph harm

In [ ]:
lmax=6
Cl2dOm_08 = anis.sph_harm_from_map(moll2dOm_tt08, lmax=6)
Cl2dOm_16 = anis.sph_harm_from_map(moll2dOm_tt16, lmax=6)
Cl2dOm_32 = anis.sph_harm_from_map(moll2dOm_tt32, lmax=6)
Cl2dOm_64 = anis.sph_harm_from_map(moll2dOm_tt64, lmax=6)
# Cl2dOm_128 = anis.sph_harm_from_map(moll2dOm_tt128, lmax=6)

In [ ]:
Cl2dOm_arrs = np.array([Cl2dOm_08, Cl2dOm_16, Cl2dOm_32, Cl2dOm_64]) #, Cl2dOm_128])
print(Cl2dOm_arrs.shape)

In [ ]:
print(nsides)

In [ ]:
rr = 0
ll=1

xx = np.arange(lmax+1)
colors = cm.rainbow(np.linspace(0,1,len(Cl2dOm_arrs)))

for ff in np.array([1,5,10]):
    yy = Cl2dOm_arrs
    fig, ax = plot.figax(xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')
    for ii, nside in enumerate(nsides):
        ax.plot(xx, yy[ii,ff,rr,:], color=colors[ii], label=('$\ell=%d$, $\ell_\mathrm{max}=%d,$ nside=%d' % (ll, lmax, nside)))
    ax.legend()
    ax.set_title('Sample %d, Realization %d, $f$=%.2f/yr, $h_c^2/d\Omega$' % (nn, rr,  fobs[ff]*YR, ))
    fig.tight_layout()

In [ ]:
ll=1

xx = fobs*YR
colors = cm.rainbow(np.linspace(0,1,len(Cl2dOm_arrs)))

for ll in range(1,7):
    yy = Cl2dOm_arrs[...,ll]/Cl2dOm_arrs[...,0]
    fig, ax = plot.figax(xlabel='$f$ yr$^{-1}$', ylabel='$C_1/C_0$')
    for ii, nside in enumerate(nsides):
        ax.plot(xx, np.median(yy[ii], axis=-1), color=colors[ii], label=('$\ell=%d$, $\ell_\mathrm{max}=%d,$ nside=%d' % (ll, lmax, nside)))
        for pp in [50,]:
            percs = pp/2
            percs = [50-percs, 50+percs]
            ax.fill_between(xx, *np.percentile(yy[ii], percs, axis=-1), alpha=0.25, color=colors[ii])

    ax.legend()
    ax.set_title('Sample %d, $\ell$=%d, $h_c^2/d\Omega$' % (nn,ll))
    fig.tight_layout()

## 3.3 Save npz

In [ ]:
save_loc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/healpix/varying_nside_testnpz'
np.savez(save_loc+'/hc2dOm_09B.npz', 
         pix_ss08=pix_ss08, pix_ss16=pix_ss16, pix_ss32=pix_ss32, pix_ss64=pix_ss64, #pix_ss128=pix_ss128,
         moll2dOm_ss08=moll2dOm_ss08, moll2dOm_ss16=moll2dOm_ss16, moll2dOm_ss32=moll2dOm_ss32, moll2dOm_ss64=moll2dOm_ss64, #moll2dOm_ss128=moll2dOm_ss128,
         moll2dOm_bg08=moll2dOm_bg08, moll2dOm_bg16=moll2dOm_bg16, moll2dOm_bg32=moll2dOm_bg32, moll2dOm_bg64=moll2dOm_bg64,# moll_bg128=moll_bg128,
         moll2dOm_tt08=moll2dOm_tt08, moll2dOm_tt16=moll2dOm_tt16, moll2dOm_tt32=moll2dOm_tt32, moll2dOm_tt64=moll2dOm_tt64, #moll_tt128=moll_tt128,
         Cl2dOm_arrs=Cl2dOm_arrs, nsides=nsides)
        #  Cl_08=Cl_08, Cl_16=Cl_16, Cl_32=Cl_32, Cl_64=Cl_64, Cl_128=Cl_128)